# <center>Sentiment Analysis on Harvard University Reviews</center>

** **
## <center>*03 - Rating Prediction Notebook*</center>

This third and final notebook is an extention of the initial scope of the project where it will be attempted to make predictions about the rating variable based on the text review. We will start by applying the necessary Text Preprocessing, followed by training models and assessing their results. 

To finalise this notebook, Future Improvements of this notebook will be stated, as well as a final conclusion of the project.

** **

This project was developed by <br><br>

*<center>António Oliveira - 2023039 - Industrial Applications of AI*

# Table of Contents

1. [Importing Data & Libraries](#1.-Importing-Data-&-Libraries)

    1.1 [Libraries](#1.1-Libraries)
    
    1.2 [Data](#1.2-Importing-Data) <br><br>
    
2. [Text Preprocessing](#2.-Text-Preprocessing)

    2.1 [Vectorization](#2.1-Vectorization) <br><br>
    
3. [Model Training](#3.-Model-Training)<br><br>

4. [Future Improvements](#4.-Future-Improvements)<br><br>

5. [Conclusion](5.-Conclusion) <br><br>

6. [AI Statement](#6.-AI-Statement)

## 1. Importing Libraries & Data

### 1.1 Libraries

In [8]:
import pandas as pd
import numpy as np

# stopwords
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

# Vectorization
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from gensim.models.doc2vec import Doc2Vec

# Random Search, K Fold
from sklearn.model_selection import RandomizedSearchCV, ParameterGrid, StratifiedKFold

# Models
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

# Model Evaluation
from sklearn.metrics import classification_report


# external py file
import functions2 as ext

# Warnings
import warnings
warnings.filterwarnings("ignore")

Checking the available functions in the external file.

In [2]:
functions = [func for func in dir(ext) if callable(getattr(ext, func))]

# Print the list of functions
print("Functions in the external module:\n")
for func in functions:
    print(func+ '\n')

Functions in the external module:

Doc2Vec

PorterStemmer

TaggedDocument

WordNetLemmatizer

contains_emoji

detect

detect_language

detect_outliers_per_column

is_emoji

plot_boxplots

preprocessor

read_corpus

replace_emojis

sent_tokenize

stopword_remover

translate_with_deepl

word_tokenize



### 1.2 Data

In [3]:
data = pd.read_csv('/Users/antoniooliveira/Projects/Industrial Applications of AI/Assignment 4/clean_data_v2.csv')
data.head(3)

,published_date,published_platform,rating,helpful_votes,title,text,Hour,Day,Month,Year,...,language,char_count,translated_text,contains_emoji,title_contains_emoji,no_emoji_text,clean_text,translated_title,no_emoji_title,clean_title
0,2023-12-28 13:02:14+00:00,Mobile,5,0,Best classes and good environment,Good thanks for everything good work group 👍 h...,3,28,12,2023,...,en,124,Good thanks for everything good work group 👍 h...,True,False,Good thanks for everything good work group thu...,Good thanks everything good work group thumbs_...,Best classes and good environment,Best classes and good environment,Best classes good environment
1,2023-12-12 05:38:26+00:00,Desktop,4,0,Harvard University,Harvard University was founded in 1636 and is ...,1,12,12,2023,...,en,322,Harvard University was founded in 1636 and is ...,False,False,Harvard University was founded in 1636 and is ...,Harvard University founded 1636 private Ivy Le...,Harvard University,Harvard University,Harvard University
2,2023-12-10 13:21:35+00:00,Mobile,3,0,Walk around campus,We did a walk around most of the Harvard Campu...,6,10,12,2023,...,en,301,We did a walk around most of the Harvard Campu...,False,False,We did a walk around most of the Harvard Campu...,"We walk around Harvard Campus, beautiful old, ...",Walk around campus,Walk around campus,Walk around campus


Dropping columns unnecessary to this task.

In [4]:
data = data.drop(['title', 'text', 'contains_emoji',
                 'title_contains_emoji', 'translated_text',
                 'clean_title', 'translated_title', 'Timezone'], axis = 1)

Since this notebook consists of an extention of the project, the final preprocessing steps will be done here. These will take advantage of the text already translated and without any emojis, and apply the *preprocessor* function (available in *functions2*) to the *no_emoji_text*.

With this we aim to experiment with lemmatisation, stemming or none, and after compare the results of different models by assessing which performs best

## 2. Text Preprocessing

In these first steps, various Word Reductions techniques will be applied, creating a new column in the existing dataframe for each. The first one, *Lemmatization* consists of reducing a word to its lemma. The other one, stemming, consists of reducing a word to its stem. These are similar techniques, therefore an example follows for better understanding.

![abdhwie](https://miro.medium.com/v2/resize:fit:1400/0*6k_6zouDWBMWkehE)

In addition to these techniques, there was also a column create without any word reduction techniques.

**Lemmatization**

In [5]:
data['lemmatized_text'] = data['no_emoji_text'].apply(lambda text: ext.preprocessor(text,
                 remove_punctuation=False,
                 lowercase=True,
                 tokenized_output=False,
                 remove_stopwords=True,
                 lemmatization=True,
                 stemming=False,
                 sentence_output=False
))

                         
data['lemmatized_text']

0       good thanks everything good work group thumbs_...
1       harvard university founded 1636 private ivy le...
2       walk around harvard campus, beautiful old, exp...
3       walk university ground tour guide- tour cruise...
4       finally made harvard!! iconic university campu...
                              ...                        
3144    boston usually time go cambridge( subway) sudd...
3145    hoped harvard university tour would give u ins...
3146    see city, study home one world's+ emblazoned u...
3147    hello, studied harvard law school year therefo...
3148    lovely building long history. free tour( engli...
Name: lemmatized_text, Length: 3149, dtype: object

**Stemming**

In [6]:
data['stemmed_text'] = data['no_emoji_text'].apply(lambda text: ext.preprocessor(text,
                 remove_punctuation=False,
                 lowercase=True,
                 tokenized_output=False,
                 remove_stopwords=True,
                 lemmatization=False,
                 stemming=True,
                 sentence_output=False
))

                         
data['stemmed_text']

0       good thank everyth good work group thumbs_up h...
1       harvard univers found 1636 privat ivi leagu re...
2       walk around harvard campu, beauti old, expect ...
3       walk univers ground tour guid- tour cruis ship...
4       final made harvard!! icon univers campu worth ...
                              ...                        
3144    boston usual time go cambridg( subway) suddenl...
3145    hope harvard univers tour would give us insigh...
3146    see citi, studi home one world's+ emblazon uni...
3147    hello, studi harvard law school year therefor ...
3148    love build long histori. free tour( english) o...
Name: stemmed_text, Length: 3149, dtype: object

**Simple**

In [7]:
data['simple_text'] = data['no_emoji_text'].apply(lambda text: ext.preprocessor(text,
                 remove_punctuation=False,
                 lowercase=True,
                 tokenized_output=False,
                 remove_stopwords=True,
                 lemmatization=False,
                 stemming=False,
                 sentence_output=False
))

                         
data['simple_text']

0       good thanks everything good work group thumbs_...
1       harvard university founded 1636 private ivy le...
2       walk around harvard campus, beautiful old, exp...
3       walk university grounds tour guide- tour cruis...
4       finally made harvard!! iconic university campu...
                              ...                        
3144    boston usually time go cambridge( subway) sudd...
3145    hoped harvard university tour would give us in...
3146    see city, study home one world's+ emblazoned u...
3147    hello, studied harvard law school year therefo...
3148    lovely building long history. free tours( engl...
Name: simple_text, Length: 3149, dtype: object

### 2.1 Vectorization

As we are using different techniques to reduce words (Stemming, Lemmatization and None), we must define in which column the Vectorization step is to be performed.

In terms of Vectorization Techniques, a brief description of the used ones follows:

- Bag of Words - simple text vectorisation that does not consider word order or context

- TF-IDF - gives more weight to less common words (can be missleading)

- Word2Vec - produces dense word embbedings that capture semantic meanings and relationships between words (more complex)

In [61]:
#column = data['stemmed_text']
column = data['simple_text']
#column = data['lemmatized_text']

**Bag Of Words**

In [52]:
vectorizer = CountVectorizer(stop_words=stop_words,max_features=2500)
corpus = column
X = vectorizer.fit_transform(corpus)

feature_names = vectorizer.get_feature_names_out()
print("Selected Features:", feature_names)


Selected Features: ['00' '000' '10' ... 'youth' 'youthful' 'zuckerberg']


**TF-IDF**

In [57]:
vectorizer = TfidfVectorizer(stop_words=stop_words,max_features=2500)
corpus = column
X = vectorizer.fit_transform(corpus)

feature_names = vectorizer.get_feature_names_out()
print("Selected Features:", feature_names)

Selected Features: ['00' '000' '10' ... 'youth' 'youthful' 'zuckerberg']


**Document Embeddings (Doc2Vec)**

In [62]:
corpus = list(ext.read_corpus(column=column))

Initialising the model and building its vocabolary.

In [63]:
model = Doc2Vec(vector_size=300,negative=5, hs=0, min_count=2,dm=0, sample = 0,epochs=30,workers = 8)
model.build_vocab(corpus)

Showing the total number of words in the corpus.

In [64]:
model.corpus_total_words

656180

Trainining the model.

In [65]:
model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs)

Storing the document vectors as X, as it was done previously when applying BoW and TF-IDF.

In [66]:
X = []
for i in range(len(model.dv)):
    X.append(model.dv[i])

In [67]:
X = np.array(X)

## 3. Model Training

In this section different models will be trained with the vectorised data. This process was developed as follows:
1. Selection of a Word Reduction Technique (Lemmatization, Stemming, None)
2. Selection of a Vectorization Technique (BoW, TF-IDF, Doc2Vec)
3. Model Training and Hyperparameter Tuning

For this reason, it must be noted that since when running the Vectorization Techniques they overwrite each other, only one can be ran at a time. As so, all but the best one will be left commented.

After this selection, a baseline model was produced, to be used as a guideline when evaluating different models. This baseline model was produced using Stratified K-Fold and Logistic Regression, without any hyperparameter tuning.

However, before starting, one must define the target.


In [11]:
y = data['rating']

**Baseline Model**

For this model it was used:
- Lemmatization
- BoW

Where a weighted average of 0.47 was achieved.

In [159]:
rf = RandomForestClassifier()

# Initialize Stratified K-Fold
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(stratified_kfold.split(X, y)):
    # Extract the training and validation data for this fold
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # Train the model 
    rf.fit(X_train, y_train)

    # Make predictions 
    predictions = rf.predict(X_val)

    # Evaluate the model
    print(f"Fold {fold + 1}:")
    print(classification_report(y_val, predictions))


Fold 1:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00         9
           3       0.00      0.00      0.00        52
           4       0.46      0.17      0.25       200
           5       0.61      0.92      0.73       366

    accuracy                           0.59       630
   macro avg       0.21      0.22      0.20       630
weighted avg       0.50      0.59      0.51       630

Fold 2:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00         9
           3       1.00      0.02      0.04        52
           4       0.39      0.10      0.17       200
           5       0.59      0.93      0.72       366

    accuracy                           0.57       630
   macro avg       0.40      0.21      0.18       630
weighted avg       0.55      0.57      0.47       630

Fold 3

After Looking at the baseline model, it is clear that the chosen model is much better when predicting a high rating (4,5) than a lower one (1,2). This would be expected, since the number of observations with a low rating is very small.

Next several models will be trained, aiming to improve the weighted average and f1-scores for all ratings. This will be done by using *Stratified K-Fold+ to perform the train-test split, where 5 folds will be used. The hyperparameter tunnning will be done with *Random Search*, as it is computationally less expensive and faster than *Grid Search*.

**Multinomial NB**

MultinomialNB only works when X doesn't include negative values. Therefore it wasn't applied when X resulted from the document embeddings (Doc2vec).

In [76]:
# Defining Parameters

param_dist = {
    'alpha': np.linspace(0, 1, 20),
    'fit_prior': [True, False],
    'force_alpha': [True, False]
}

# Perform Random Search
random_search = RandomizedSearchCV(MultinomialNB(), param_distributions=param_dist, n_iter=50, cv=5, scoring='f1_weighted', n_jobs=-1, random_state=42)
random_search.fit(X, y)

# Get the best parameters
best_params = random_search.best_params_

/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated 

/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:61

/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated 

/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated 

In [77]:
print(f"Best parameters: {best_params}")

Best parameters: {'force_alpha': True, 'fit_prior': True, 'alpha': 0.7368421052631579}


In [78]:
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Iterate over the folds
for fold, (train_idx, val_idx) in enumerate(stratified_kfold.split(X, y)):
    
    # Extract the training and testing data for this fold
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # Train and evaluate your model on this fold
    print(f"Fold {fold + 1}:")
    print("Training data:", X_train.shape[0], len(y_train))
    print("Testing data:", X_val.shape[0], len(y_val))
    
    # Training and evaluation 
    mnb = MultinomialNB(**best_params)
    mnb.fit(X_train,y_train)
    predictions = mnb.predict(X_val)
    print(classification_report(y_val, predictions))
    print("\n")

Fold 1:
Training data: 2519 2519
Testing data: 630 630
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00         9
           3       0.36      0.31      0.33        52
           4       0.45      0.42      0.43       200
           5       0.70      0.76      0.73       366

    accuracy                           0.60       630
   macro avg       0.30      0.30      0.30       630
weighted avg       0.58      0.60      0.59       630



Fold 2:
Training data: 2519 2519
Testing data: 630 630
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         3
           2       0.20      0.11      0.14         9
           3       0.47      0.31      0.37        52
           4       0.42      0.39      0.40       200
           5       0.67      0.74      0.71       366

    accuracy                           0.58       630
   macro avg       

**<center>Results in the last Fold<center>** <br><br>
    

| Model                       | Word Reduction | Vectorization | Weighted Avg | Parameters |
|----------------|---------|---------------------|--------------------|--------------------|
| Multinomial NB              | Lemmatization | BoW | 0.59  | {'force_alpha': True, 'fit_prior': True, 'alpha': 0.5263157894736842} |
| Multinomial NB              | Lemmatization | TF-IDF | 0.53 | {'force_alpha': False, 'fit_prior': False, 'alpha': 0.5789473684210527} |
| Multinomial NB              | Stemming | BoW | 0.59  | {'force_alpha': True, 'fit_prior': True, 'alpha': 0.7368421052631579} |
| Multinomial NB              | Stemming | TF-IDF |  0.54  | {'force_alpha': True, 'fit_prior': False, 'alpha': 0.894736842105263} |
| Multinomial NB              | None | BoW |  0.57  | {'force_alpha': False, 'fit_prior': True, 'alpha': 0.631578947368421} |
| Multinomial NB              | None | TF-IDF | 0.53  | {'force_alpha': False, 'fit_prior': False, 'alpha': 0.5789473684210527}


**Random Forest**

In [135]:
param_grid = {
    'n_estimators': np.arange(10, 200, 10),
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None] + list(np.arange(5, 30, 5)),
    'min_samples_split': np.arange(2, 10, 1),
    'min_samples_leaf': np.arange(1, 10, 1),
    'bootstrap': [True, False]
}

# Perform Random Search
random_search = RandomizedSearchCV(RandomForestClassifier(random_state=42), param_distributions=param_grid, n_iter=50, cv=5, scoring='f1_weighted', n_jobs=-1, random_state=42)
random_search.fit(X, y)

# Get the best parameters
best_params = random_search.best_params_

/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated 

/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated 

/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated 

/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated 

/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated 

...

/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/antoniooliveira/anaconda

/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated 

/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated 

/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated 

KeyboardInterrupt: 

In [ ]:
print(f"Best parameters: {best_params}")

In [ ]:
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Iterate over the folds
for fold, (train_idx, val_idx) in enumerate(stratified_kfold.split(X, y)):
    
    # Extract the training and testing data for this fold
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # Train and evaluate your model on this fold
    print(f"Fold {fold + 1}:")
    print("Training data:", X_train.shape[0], len(y_train))
    print("Testing data:", X_val.shape[0], len(y_val))
    
    # Training and evaluation 
    rf = RandomForestClassifier(**best_params)
    rf.fit(X_train,y_train)
    predictions = rf.predict(X_val)
    print(classification_report(y_val, predictions))
    print("\n")

**<center>Results in the last Fold<center>** <br><br>



| Model                       | Word Reduction | Vectorization | Weighted Avg | Parameters |
|-----------------------------|---------|---------------------|--------------------|--------------------|
| Random Forest              | Lemmatization | BoW | 0.50  | {'n_estimators': 70, 'min_samples_split': 8, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': None, 'bootstrap': False}
| Random Forest              | Lemmatization | TF-IDF | 0.48   | {'n_estimators': 70, 'min_samples_split': 8, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': None, 'bootstrap': False} |
| Random Forest              | Lemmatization | Doc2Vec | 0.50  | {'n_estimators': 20, 'min_samples_split': 4, 'min_samples_leaf': 5, 'max_features': 'sqrt', 'max_depth': 20, 'bootstrap': True} |
| Random Forest              | Stemming | BoW | 0.48  | {'n_estimators': 70, 'min_samples_split': 8, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': None, 'bootstrap': False} |
| Random Forest              | Stemming | TF-IDF | 0.47  | {'n_estimators': 70, 'min_samples_split': 8, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': None, 'bootstrap': False} |
| Random Forest              | Stemming | Doc2Vec | 0.47  | {'n_estimators': 50, 'min_samples_split': 9, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 20, 'bootstrap': False}
| Random Forest              | None | BoW | 0.51  | {'n_estimators': 190, 'min_samples_split': 3, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': None, 'bootstrap': True} |
| Random Forest              | None | TF-IDF | 0.51  |  {'n_estimators': 70, 'min_samples_split': 8, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': None, 'bootstrap': False}
| Random Forest              | None | Doc2Vec | 0.48  | {'n_estimators': 20, 'min_samples_split': 4, 'min_samples_leaf': 5, 'max_features': 'sqrt', 'max_depth': 20, 'bootstrap': True}


**Logistic Regression**

In [177]:
param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'C': np.logspace(-4, 4, 50),
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'l1_ratio': np.linspace(0, 1, 20)
}

# Perform Random Search
random_search = RandomizedSearchCV(LogisticRegression(random_state=42), param_distributions=param_grid, n_iter=50, cv=5, scoring='f1_weighted', n_jobs=-1, random_state=42)
random_search.fit(X, y)

# Get the best parameters
best_params = random_search.best_params_

/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=none)
  warnings.warn(
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=none)
  warnings.warn(
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/pyth

/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_

/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_

/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_

/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated 

...

/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated 

/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated 

/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated 

/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated 

/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated 

In [178]:
print(f"Best parameters: {best_params}")

Best parameters: {'solver': 'saga', 'penalty': 'l2', 'l1_ratio': 0.42105263157894735, 'C': 10000.0}


In [179]:
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Iterate over the folds
for fold, (train_idx, val_idx) in enumerate(stratified_kfold.split(X, y)):
    
    # Extract the training and testing data for this fold
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # Train and evaluate your model on this fold
    print(f"Fold {fold + 1}:")
    print("Training data:", X_train.shape[0], len(y_train))
    print("Testing data:", X_val.shape[0], len(y_val))
    
    # Training and evaluation 
    lr = LogisticRegression(**best_params)
    lr.fit(X_train,y_train)
    predictions = lr.predict(X_val)
    print(classification_report(y_val, predictions))
    print("\n")

Fold 1:
Training data: 2519 2519
Testing data: 630 630
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00         9
           3       0.00      0.00      0.00        52
           4       0.49      0.20      0.29       200
           5       0.61      0.91      0.73       366

    accuracy                           0.59       630
   macro avg       0.22      0.22      0.20       630
weighted avg       0.51      0.59      0.52       630



Fold 2:
Training data: 2519 2519
Testing data: 630 630
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00         9
           3       0.00      0.00      0.00        52
           4       0.38      0.18      0.25       200
           5       0.60      0.87      0.71       366

    accuracy                           0.56       630
   macro avg       

**<center>Results in the last Fold<center>** <br><br>
    

| Model                       | Word Reduction | Vectorization | Weighted Avg | Parameters |
|----------------|---------|---------------------|--------------------|--------------------|
| Logistic Regression              | Lemmatization | BoW | 0.58  | {'solver': 'saga', 'penalty': 'l1', 'l1_ratio': 0.21052631578947367, 'C': 3.727593720314938} |
| Logistic Regression              | Lemmatization | TF-IDF | 0.57 | {'solver': 'saga', 'penalty': 'elasticnet', 'l1_ratio': 0.47368421052631576, 'C': 3.727593720314938} |
| Logistic Regression              | Lemmatization | Doc2Vec | 0.50 | {'solver': 'saga', 'penalty': 'l2', 'l1_ratio': 0.42105263157894735, 'C': 10000.0} |
| Logistic Regression              | Stemming | BoW | 0.57  | {'solver': 'saga', 'penalty': 'elasticnet', 'l1_ratio': 0.47368421052631576, 'C': 3.727593720314938} |
| Logistic Regression              | Stemming | TF-IDF |  0.57  | {'solver': 'saga', 'penalty': 'elasticnet', 'l1_ratio': 0.47368421052631576, 'C': 3.727593720314938} |
| Logistic Regression              | Stemming | Doc2Vec |  0.50  |  {'solver': 'liblinear', 'penalty': 'l1', 'l1_ratio': 0.2631578947368421, 'C': 75.43120063354607} |
| Logistic Regression              | None | BoW |  0.57  | {'solver': 'saga', 'penalty': 'l1', 'l1_ratio': 0.21052631578947367, 'C': 3.727593720314938} |
| Logistic Regression              | None | TF-IDF | 0.57  | {'solver': 'saga', 'penalty': 'elasticnet', 'l1_ratio': 0.47368421052631576, 'C': 3.727593720314938}
| Logistic Regression              | None | Doc2Vec | 0.49  | {'solver': 'saga', 'penalty': 'l2', 'l1_ratio': 0.42105263157894735, 'C': 10000.0}


**Support Vector Machine**

In [68]:
param_grid = {
    'C': np.logspace(-4, 4, 50),
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': [2, 3, 4, 5],
    'gamma': ['scale', 'auto'],
    'coef0': np.linspace(0, 1, 10)
}

# Perform Random Search
random_search = RandomizedSearchCV(SVC(random_state=42), param_distributions=param_grid, n_iter=50, cv=5, scoring='f1_weighted', n_jobs=-1, random_state=42)
random_search.fit(X, y)

# Get the best parameters
best_params = random_search.best_params_

/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated 

/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated 

/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated 

/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated 

/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated 

...

/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated 

/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated 

/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated 

/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated 

/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/antoniooliveira/anaconda3/envs/Text_Mining/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated 

In [69]:
print(f"Best parameters: {best_params}")

Best parameters: {'kernel': 'rbf', 'gamma': 'scale', 'degree': 5, 'coef0': 1.0, 'C': 3.727593720314938}


In [70]:
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Iterate over the folds
for fold, (train_idx, val_idx) in enumerate(stratified_kfold.split(X, y)):
    
    # Extract the training and testing data for this fold
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # Train and evaluate your model on this fold
    print(f"Fold {fold + 1}:")
    print("Training data:", X_train.shape[0], len(y_train))
    print("Testing data:", X_val.shape[0], len(y_val))
    
    # Training and evaluation 
    svc = SVC(**best_params)
    svc.fit(X_train,y_train)
    predictions = svc.predict(X_val)
    print(classification_report(y_val, predictions))
    print("\n")

Fold 1:
Training data: 2519 2519
Testing data: 630 630
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00         9
           3       0.20      0.02      0.04        52
           4       0.41      0.29      0.34       200
           5       0.63      0.83      0.72       366

    accuracy                           0.58       630
   macro avg       0.25      0.23      0.22       630
weighted avg       0.51      0.58      0.53       630



Fold 2:
Training data: 2519 2519
Testing data: 630 630
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00         9
           3       0.40      0.04      0.07        52
           4       0.32      0.22      0.26       200
           5       0.59      0.78      0.67       366

    accuracy                           0.53       630
   macro avg       

**<center>Results in the last Fold<center>** <br><br>
    

| Model                       | Word Reduction | Vectorization | Weighted Avg | Parameters |
|----------------|---------|---------------------|--------------------|--------------------|
| SVC              | Lemmatization | BoW | 0.58  | {'kernel': 'linear', 'gamma': 'auto', 'degree': 5, 'coef0': 1.0, 'C': 0.12648552168552957} |
| SVC              | Lemmatization | TF-IDF | 0.58 | {'kernel': 'poly', 'gamma': 'auto', 'degree': 5, 'coef0': 0.7777777777777777, 'C': 4714.8663634573895} |
| SVC              | Lemmatization | Doc2Vec | 0.48 | {'kernel': 'rbf', 'gamma': 'scale', 'degree': 5, 'coef0': 1.0, 'C': 3.727593720314938} |
| SVC              | Stemming | BoW | 0.57  | {'kernel': 'linear', 'gamma': 'auto', 'degree': 5, 'coef0': 1.0, 'C': 0.12648552168552957} |
| SVC              | Stemming | TF-IDF |  0.55  | {'kernel': 'rbf', 'gamma': 'scale', 'degree': 3, 'coef0': 1.0, 'C': 494.1713361323828} |
| SVC              | Stemming | Doc2Vec |  0.50  |  {'kernel': 'rbf', 'gamma': 'scale', 'degree': 5, 'coef0': 1.0, 'C': 3.727593720314938} |
| SVC              | None | BoW |  0.59  | {'kernel': 'linear', 'gamma': 'auto', 'degree': 5, 'coef0': 1.0, 'C': 0.12648552168552957} |
| SVC              | None | TF-IDF | 0.56  | {'kernel': 'poly', 'gamma': 'auto', 'degree': 5, 'coef0': 0.7777777777777777, 'C': 4714.8663634573895}
| SVC              | None | Doc2Vec | 0.50  | {'kernel': 'rbf', 'gamma': 'scale', 'degree': 5, 'coef0': 1.0, 'C': 3.727593720314938}


** **

In this section, several models were implemented, in an attempt to predict the *Rating*. It can be concluded that the most accurate model is **Support Vector Machine (SVM)**, when using Bag of Words Vectorization and no Word Reduction Techniques. It achieved a weighted Avg of **0.59** on the validation set. 

It can be seen that, as the number of folds increases, sometimes the overall weighted average decreases. This happens because the models are sacrificing their performance when predicting higher ratings (4,5), in order to try to predict the rest of the possible ratings. An example of this can be seen in the implementation of Logistic Regression when using Doc2Vec and no Word Reduction techniques. The first fold starts with a Weighted Avg of 0.52 and f1-scores of 0.29 for rating 4, 0.73 for rating 5, and 0 for the rest. However, in the last fold, the model is already able to predict rating 3 accuratly 3% of the times. As a consequence, the f1-scores of rating 4 and 5 decreased to 0.23 and 0.72 respectively, along with the overall final weighted average, to 0.49.

All the used models showed difficulty when predicting Ratings 1 and 2, even when using Stratified K-Fold for the data train-test split. This problem arises from the disparity between the number of observations collected for ratings 3, 4 and 5 and ratings 1 and 2.

## 4. Future Improvements

In the future it would be possible to improve the outcome of these models by experimenting with oversampling or undersampling techniques. If enough data was available, Deep Learning Models, could also be used, along with pre-trained models, which could be fine-tuned for the problem in-hand.

## 5. Conclusion

This project's initial aim was to understand the Sentiment Transmited by Reviews made about Harvard University. To do so, a first notebook was developed, which contained EDA and text preprocessing steps. Next, a Sentiment Analysis notebook was created, were different Lexicon-Based Sentiment Analysers were implemented, and conclusions about their results were taken.

Since the results of the second notebook did not meet expectations, a decision was made to extend the project's aimd to include the prediction of the target *rating* by using the text Reviews. This was done in the third notebook, which consisted of the necessary steps to achieve this goal.  

It can concluded that this project was overall a success, even if the Sentiment Analysis section did not yield major conclusions, as the Rating prediction section allowed us to achieve satisfatory results. It must be stated that both the Sentiment Analysis and Rating Prediction were majorly influenced by the lack of negative observations, which not only makes it hard to take conclusions from SA, but also increases the difficulty for models to accuratly make predictions of the lower ratings.

Since this is the case, it can be believed that generally, people are Satisfied with Harvard University.

** **

## 6. AI Statement

In this project AI was applied to help with the text translation, namely with the function *translate_with_deepl*, available in the *function2* file.

It was also used to perform emoji detection and replacement.

However, all functions were adapted to output exactly what was needed, in a clear and understandable way.